Demonstrate how GENERATE\_SERIES works:

In [1]:
DECLARE @String NVARCHAR(MAX) = 'A<>BC<>DEF<>GHIJ<>KLMNO', @Delimiter NVARCHAR(10)='<>';

SELECT	value, @String AS String, @Delimiter AS Delimiter, LEN(@String) AS String_Len, LEN(@Delimiter) AS Delimiter_Len
FROM	GENERATE_SERIES(CAST(1 AS BIGINT), CAST(ISNULL(LEN(@String),0) AS BIGINT));

(23 rows affected)

Total execution time: 00:00:00.043

value String Delimiter String_Len Delimiter_Len 1 A<>BC<>DEF<>GHIJ<>KLMNO <> 23 2 2 A<>BC<>DEF<>GHIJ<>KLMNO <> 23 2 3 A<>BC<>DEF<>GHIJ<>KLMNO <> 23 2 4 A<>BC<>DEF<>GHIJ<>KLMNO <> 23 2 5 A<>BC<>DEF<>GHIJ<>KLMNO <> 23 2 6 A<>BC<>DEF<>GHIJ<>KLMNO <> 23 2 7 A<>BC<>DEF<>GHIJ<>KLMNO <> 23 2 8 A<>BC<>DEF<>GHIJ<>KLMNO <> 23 2 9 A<>BC<>DEF<>GHIJ<>KLMNO <> 23 2 10 A<>BC<>DEF<>GHIJ<>KLMNO <> 23 2 11 A<>BC<>DEF<>GHIJ<>KLMNO <> 23 2 12 A<>BC<>DEF<>GHIJ<>KLMNO <> 23 2 13 A<>BC<>DEF<>GHIJ<>KLMNO <> 23 2 14 A<>BC<>DEF<>GHIJ<>KLMNO <> 23 2 15 A<>BC<>DEF<>GHIJ<>KLMNO <> 23 2 16 A<>BC<>DEF<>GHIJ<>KLMNO <> 23 2 17 A<>BC<>DEF<>GHIJ<>KLMNO <> 23 2 18 A<>BC<>DEF<>GHIJ<>KLMNO <> 23 2 19 A<>BC<>DEF<>GHIJ<>KLMNO <> 23 2 20 A<>BC<>DEF<>GHIJ<>KLMNO <> 23 2 21 A<>BC<>DEF<>GHIJ<>KLMNO <> 23 2 22 A<>BC<>DEF<>GHIJ<>KLMNO <> 23 2 23 A<>BC<>DEF<>GHIJ<>KLMNO <> 23 2

Calculate the segment start and filter only for where delimiters are:

In [2]:
DECLARE @String NVARCHAR(MAX) = 'A<>BC<>DEF<>GHIJ<>KLMNO', @Delimiter NVARCHAR(10)='<>';

SELECT	value, @String AS String, @Delimiter AS Delimiter,
		LEN(@Delimiter) AS Delimiter_Len, value + LEN(@Delimiter) AS Segment_Start
FROM	GENERATE_SERIES(CAST(1 AS BIGINT), CAST(ISNULL(LEN(@String),0) AS BIGINT))
WHERE	SUBSTRING(@String, value, LEN(@Delimiter)) = @Delimiter;

(4 rows affected)

Total execution time: 00:00:00.009

value String Delimiter Delimiter_Len Segment_Start 2 A<>BC<>DEF<>GHIJ<>KLMNO <> 2 4 6 A<>BC<>DEF<>GHIJ<>KLMNO <> 2 8 11 A<>BC<>DEF<>GHIJ<>KLMNO <> 2 13 17 A<>BC<>DEF<>GHIJ<>KLMNO <> 2 19

Get the start of the next segment:

In [3]:
DECLARE @String NVARCHAR(MAX) = 'A<>BC<>DEF<>GHIJ<>KLMNO', @Delimiter NVARCHAR(10)='<>';

SELECT	value, @String AS String, @Delimiter AS Delimiter,
		LEN(@Delimiter) AS Delimiter_Len, value + LEN(@Delimiter) AS Segment_Start,
		LEAD(value, 1) OVER (ORDER BY (SELECT 1))
FROM	GENERATE_SERIES(CAST(1 AS BIGINT), CAST(ISNULL(LEN(@String),0) AS BIGINT))
WHERE	SUBSTRING(@String, value, LEN(@Delimiter)) = @Delimiter;

(4 rows affected)

Total execution time: 00:00:00.009

value String Delimiter Delimiter_Len Segment_Start (No column name) 2 A<>BC<>DEF<>GHIJ<>KLMNO <> 2 4 6 6 A<>BC<>DEF<>GHIJ<>KLMNO <> 2 8 11 11 A<>BC<>DEF<>GHIJ<>KLMNO <> 2 13 17 17 A<>BC<>DEF<>GHIJ<>KLMNO <> 2 19 NULL

Resolve value for next segment on last row:

In [4]:
DECLARE @String NVARCHAR(MAX) = 'A<>BC<>DEF<>GHIJ<>KLMNO', @Delimiter NVARCHAR(10)='<>';

SELECT	value, @String AS String, @Delimiter AS Delimiter,
		LEN(@Delimiter) AS Delimiter_Len, value + LEN(@Delimiter) AS Segment_Start,
		LEAD(value, 1, LEN(@String)+LEN(@Delimiter)+1) OVER (ORDER BY (SELECT 1)) AS Next_Segment
FROM	GENERATE_SERIES(CAST(1 AS BIGINT), CAST(ISNULL(LEN(@String),0) AS BIGINT))
WHERE	SUBSTRING(@String, value, LEN(@Delimiter)) = @Delimiter;

(4 rows affected)

Total execution time: 00:00:00.007

value String Delimiter Delimiter_Len Segment_Start Next_Segment 2 A<>BC<>DEF<>GHIJ<>KLMNO <> 2 4 6 6 A<>BC<>DEF<>GHIJ<>KLMNO <> 2 8 11 11 A<>BC<>DEF<>GHIJ<>KLMNO <> 2 13 17 17 A<>BC<>DEF<>GHIJ<>KLMNO <> 2 19 26

Resolve case for first segment:

In [5]:
DECLARE @String NVARCHAR(MAX) = 'A<>BC<>DEF<>GHIJ<>KLMNO', @Delimiter NVARCHAR(10)='<>';

WITH cte_segment(n, slen, dlen, spos) AS (
	SELECT	1-LEN(@Delimiter) AS value, LEN(@String), LEN(@Delimiter), 1
	UNION ALL
	SELECT	value, LEN(@String), LEN(@Delimiter), value + LEN(@Delimiter)
	FROM	GENERATE_SERIES(CAST(1 AS BIGINT), CAST(ISNULL(LEN(@String),0) AS BIGINT))
	WHERE	SUBSTRING(@String, value, LEN(@Delimiter)) = @Delimiter
	)
SELECT	n, @String AS String, @Delimiter AS Delimiter, spos AS Segment_Start,
		LEAD(spos, 1, slen + dlen + 1) OVER (ORDER BY (SELECT 1)) AS Next_Segment_Start,
		LEAD(spos, 1, slen + dlen + 1) OVER (ORDER BY (SELECT 1)) - spos - dlen AS Segment_Length
FROM	cte_segment s;

(5 rows affected)

Total execution time: 00:00:00.015

n String Delimiter Segment_Start Next_Segment_Start Segment_Length -1 A<>BC<>DEF<>GHIJ<>KLMNO <> 1 4 1 2 A<>BC<>DEF<>GHIJ<>KLMNO <> 4 8 2 6 A<>BC<>DEF<>GHIJ<>KLMNO <> 8 13 3 11 A<>BC<>DEF<>GHIJ<>KLMNO <> 13 19 4 17 A<>BC<>DEF<>GHIJ<>KLMNO <> 19 26 5

Get the segment value:

In [6]:
DECLARE @String NVARCHAR(MAX) = 'A<>BC<>DEF<>GHIJ<>KLMNO', @Delimiter NVARCHAR(10)='<>';

WITH cte_segment(n, slen, dlen, spos) AS (
	SELECT	1-LEN(@Delimiter) AS value, LEN(@String), LEN(@Delimiter), 1
	UNION ALL
	SELECT	value, LEN(@String), LEN(@Delimiter), value + LEN(@Delimiter)
	FROM	GENERATE_SERIES(CAST(1 AS BIGINT), CAST(ISNULL(LEN(@String),0) AS BIGINT))
	WHERE	SUBSTRING(@String, value, LEN(@Delimiter)) = @Delimiter
	)
SELECT	n, @String AS String, @Delimiter AS Delimiter, spos AS Segment_Start,
		LEAD(spos, 1, slen + dlen + 1) OVER (ORDER BY (SELECT 1)) AS Next_Segment_Start,
		LEAD(spos, 1, slen + dlen + 1) OVER (ORDER BY (SELECT 1)) - spos - dlen AS Segment_Length,
		SUBSTRING(@String, spos, (LEAD(spos, 1, slen + dlen + 1) OVER (ORDER BY (SELECT 1))-dlen)-spos) AS Segment
FROM	cte_segment s;

(5 rows affected)

Total execution time: 00:00:00.008

n String Delimiter Segment_Start Next_Segment_Start Segment_Length Segment -1 A<>BC<>DEF<>GHIJ<>KLMNO <> 1 4 1 A 2 A<>BC<>DEF<>GHIJ<>KLMNO <> 4 8 2 BC 6 A<>BC<>DEF<>GHIJ<>KLMNO <> 8 13 3 DEF 11 A<>BC<>DEF<>GHIJ<>KLMNO <> 13 19 4 GHIJ 17 A<>BC<>DEF<>GHIJ<>KLMNO <> 19 26 5 KLMNO

Generate the sequence number and remove extra values:

In [7]:
DECLARE @String NVARCHAR(MAX) = 'A<>BC<>DEF<>GHIJ<>KLMNO', @Delimiter NVARCHAR(10)='<>';

WITH cte_segment(n, slen, dlen, spos) AS (
	SELECT	1-LEN(@Delimiter) AS value, LEN(@String), LEN(@Delimiter), 1
	UNION ALL
	SELECT	value, LEN(@String), LEN(@Delimiter), value + LEN(@Delimiter)
	FROM	GENERATE_SERIES(CAST(1 AS BIGINT), CAST(ISNULL(LEN(@String),0) AS BIGINT))
	WHERE	SUBSTRING(@String, value, LEN(@Delimiter)) = @Delimiter
	)
SELECT	ROW_NUMBER() OVER (ORDER BY spos) AS n,
		SUBSTRING(@String, spos, (LEAD(spos, 1, slen + dlen + 1) OVER (ORDER BY (SELECT 1))-dlen)-spos) AS [value]
FROM	cte_segment s;

(5 rows affected)

Total execution time: 00:00:00.008

n value 1 A 2 BC 3 DEF 4 GHIJ 5 KLMNO